In [18]:
from data import MscDataset
import pandas as pd
from transformers import AutoTokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained('facebook/blenderbot-400M-distill')

In [20]:
def make_df(data, indices):
    human_eval_data = {'hist': [], 'query': [], 'response': []}
    for i in indices:
        idx = data.data_idx[i]
        hist = data.histories[idx]
        query = data.queries[idx]
        response = data.responses[idx]
        human_eval_data['hist'].append(hist)
        human_eval_data['query'].append(query)
        human_eval_data['response'].append(response)
    human_eval_data = pd.DataFrame(human_eval_data)
    return human_eval_data

In [21]:
test_dataset = MscDataset(
    tokenizer,
    'msc/session_5/test.txt',
    max_length=-1,
    max_session=5,
    target_session=5,
)
with open('human_eval_session5_idx.txt', 'r') as f:
    indices = [int(x.strip()) for x in f.readlines()]
    

session5_human_eval_data = make_df(test_dataset, indices)

session5_pred_base = pd.read_json('generated/final/sess5-blenderbot400M-msc.jsonl', lines=True)
session5_pred_seg1 = pd.read_json('generated/final/sess5-blenderbot-400M-RMT-seg1-mem_rleft_wleft_5_residual-textnormv3-proj-scaledv2.jsonl', lines=True)
session5_pred_seg8 = pd.read_json('generated/final/sess5-blenderbot-400M-RMT-seg8-mem_rleft_wleft_5_residual-continual-prev_sess.jsonl', lines=True)

load data:   0%|          | 0/501 [00:00<?, ?it/s]

format data:   0%|          | 0/501 [00:00<?, ?it/s]

In [22]:
test_dataset_200 = MscDataset(
    tokenizer,
    'msc/session_5/test.txt',
    max_length=-1,
    max_session=5,
    target_session=200,
    task='prev_sess',
)
with open('human_eval_session200_idx.txt', 'r') as f:
    indices = [int(x.strip()) for x in f.readlines()]
    

session200_human_eval_data = make_df(test_dataset_200, indices)

session200_pred_base = pd.read_json('generated/final/blenderbot400M-msc.jsonl', lines=True)
session200_pred_seg1 = pd.read_json('generated/final/blenderbot-400M-RMT-seg1-mem_rleft_wleft_5_residual-textnormv3-proj-scaledv2.jsonl', lines=True)
session200_pred_seg8 = pd.read_json('generated/final/blenderbot-400M-RMT-seg8-mem_rleft_wleft_5_residual-continual-prev_sess.jsonl', lines=True)

load data:   0%|          | 0/501 [00:00<?, ?it/s]

format data:   0%|          | 0/501 [00:00<?, ?it/s]

In [24]:
session5_human_eval_data['baseline'] = session5_pred_base['pred']
session5_human_eval_data['seg1'] = session5_pred_seg1['pred']
session5_human_eval_data['seg8'] = session5_pred_seg8['pred']

In [25]:
session200_human_eval_data['baseline'] = session200_pred_base['pred']
session200_human_eval_data['seg1'] = session200_pred_seg1['pred']
session200_human_eval_data['seg8'] = session200_pred_seg8['pred']

In [31]:
session5_human_eval_data['hist'] = session5_human_eval_data['hist'].apply(lambda x: '\n'.join(x))
session200_human_eval_data['hist'] = session200_human_eval_data['hist'].apply(lambda x: '\n'.join(x))

In [32]:
session5_human_eval_data.to_csv('session5_human_eval_data.csv', index=False)
session200_human_eval_data.to_csv('session200_human_eval_data.csv', index=False)